In [22]:
!pip install pdf2image jinja2


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
!apt-get update

Get:1 http://security.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:2 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://security.debian.org/debian-security bullseye-security/main amd64 Packages [269 kB]
Get:5 http://deb.debian.org/debian bullseye/main amd64 Packages [8068 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [18.8 kB]
Fetched 8564 kB in 1s (5749 kB/s)
Reading package lists... Done


In [15]:
! apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libnspr4 libnss3 libpoppler102
The following NEW packages will be installed:
  libnspr4 libnss3 libpoppler102 poppler-utils
0 upgraded, 4 newly installed, 0 to remove and 54 not upgraded.
Need to get 3283 kB of archives.
After this operation, 9559 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian bullseye/main amd64 libnspr4 amd64 2:4.29-1 [112 kB]
Get:2 http://deb.debian.org/debian bullseye/main amd64 libnss3 amd64 2:3.61-1+deb11u3 [1305 kB]
Get:3 http://deb.debian.org/debian bullseye/main amd64 libpoppler102 amd64 20.09.0-3.1+deb11u1 [1676 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 poppler-utils amd64 20.09.0-3.1+deb11u1 [189 kB]
Fetched 3283 kB in 0s (71.9 MB/s)       
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package libnspr4:a

In [2]:
import boto3
import json
from utils import parse_response, convert_image_to_bytes, render_payload, convert_pdf_to_images, get_jinja_env

In [3]:
 bedrock = boto3.client(
     service_name='bedrock-runtime',
     region_name='us-west-2'
)

In [4]:
 model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
claude_config = {
    'max_tokens': 100000,
    'temperature': 1,
    'anthropic_version': 'bedrock-2023-05-31',
    'top_p': 0.999,
    'stop_sequences': ['Human:']
}

In [9]:

images = convert_pdf_to_images("data/gfs/500_Index_Fund_Admiral__factSheet.pdf")

In [10]:
img_bytes = [convert_image_to_bytes(i) for i in images]

In [11]:

env = get_jinja_env()

In [13]:
template = env.get_template("fund_fact_sheet.txt")

print(template.render())


You are an assistant that helps perform tasks on documents. 
You have received document pages in order of read. You are to perform the task defined below. 
Do not return anything except what is asked for in the task. 


This document is a fund fact sheet. 
Your task is to extract specific values from the document as JSON format. 
You should extract these values with an <extracted_data> tag. 

These are the following data points you should extact. 
fund_name: The name of the fund
cusip: A 5 digit identifier. There may be multiple
share_class: the types of shares
ticker: The 4 character symbol for the fund
report_date: The date of the report
financial_period: The period the report covers (something like 4Q or Q4 is common, indicating a financial quarter)
document_identifier: This would denote this being a fund fact sheet
management_discussion: A paragraph describing the fund performance
performance: a table describing performance versus the benchmark. If there are time periods, break ea

In [17]:
system_prompt = 'you are an AI data assistant'
body = render_payload(img_bytes, template.render(), system_prompt, claude_config = claude_config)

In [18]:
response = bedrock.invoke_model(
    modelId=model_id,
    body=json.dumps(body))

In [19]:
result = parse_response(response)
print(result)

<extracted_data>
{
  "fund_name": "Vanguard 500 Index Fund",
  "cusip": null,
  "share_class": "Admiral Shares",
  "ticker": "VFIAX",
  "report_date": "December 31, 2022",
  "financial_period": null,
  "document_identifier": "Fund fact sheet",
  "management_discussion": "The fund employs a \"passive management\"—or indexing—investment approach designed to track the performance of the Standard & Poor's 500 Index, a widely used barometer of U.S. stock market performance that is dominated by the stocks of large U.S. companies. The fund attempts to replicate the target index by investing all, or substantially all, of its assets in the stocks that make up the index, holding each stock in approximately the same proportion as its weighting in the index.",
  "performance": {
    "Quarter": "7.56%",
    "Year to date": "-18.15%",
    "One year": "-18.15%",
    "Three years": "7.62%",
    "Five years": "9.39%",
    "Ten years": "12.52%"
  },
  "benchmark_funds": [
    "S&P 500 Index"
  ],
  "net